In [6]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

/Users/kymbradshaw/.pyenv/versions/3.10.6/envs/carbon_calculator/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
apple = Image.open('/Users/kymbradshaw/Desktop/test_images/orange.jpg')

In [20]:
processor_g = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model_g = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [21]:
inputs = processor_g(apple, return_tensors="pt")

In [22]:
with torch.no_grad():
    logits = model_g(**inputs).logits


predicted_label = logits.argmax(-1).item()
print(model_g.config.id2label[predicted_label])

orange


In [20]:
def load_model():
    """
    Loads the model from the transformers package with pretrained weights
    """
    model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

    print("✅ Model loaded")

    return model

def predict(processed_image, model):
    """
    Takes processed image and model as input and returns the class with the highest log likelihood
    """
    with torch.no_grad():
        log = model(**processed_image).logits

    predicted_label = log.argmax(-1).item()
    classification = model.config.id2label[predicted_label]

    return classification


In [21]:
def preprocessing(crop_image):
    """
    Takes image (jpeg, webp) as an input and processes it using google's image processor
    """
    image = Image.open(crop_image)
    processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
    processor(image, return_tensors='pt')
    processed_image = processor(image, return_tensors='pt')

    print("✅ Processed image")

    return processed_image

In [22]:
def prediction(crop_image):
    """
    This is the endpoint for the predictions

    Args: to be defined
    """
    processed_image = preprocessing(crop_image)
    model = load_model()
    prediction = predict(processed_image,model)
    return prediction

In [23]:
filepath = '../test_images/banana.jpg'
prediction(filepath)

✅ Processed image
✅ Model loaded


'banana'